In [1]:
 #!pip install pandas openpyxl matplotlib seaborn ipywidgets

In [10]:
#!/usr/bin/env python

# ##############################################################################
# Dashboard Dinâmico - Proposta de Cofinanciamento da APS no Mato Grosso do Sul
# Google Colab - Isa Venancio
# ##############################################################################

# Carregar pacotes
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import ipywidgets as widgets # Para criar o dashboard interativo
import matplotlib.font_manager as fm
from IPython.display import display, clear_output, HTML # Para exibir widgets e limpar saídas
# from google.colab import drive # Comentar ou remover se não for usar o Google Drive mount

# Importa o módulo statsmodels para regressão
import statsmodels.api as sm

# Definição dos PATHs dos arquivos (Célula 1)
PATH_CENSO = "/content/censo2022.csv"
PATH_MICRORREGIAO = "/content/microrregiao_ms_proposto.csv"
PATH_REPASSE_FEDERAL = "/content/pgto_federal_equipes_fev_2025.xlsx"
PATH_OBITOS = "/content/obitos_aps_2010-2023.xlsx" # NOME DO ARQUIVO CORRIGIDO AQUI!

# --- Célula 2: Carregar Dados Brutos ---

# Verifica se os arquivos existem nos caminhos esperados
if not os.path.exists(PATH_CENSO):
    print(f"ATENÇÃO: O arquivo '{PATH_CENSO}' não foi encontrado. Por favor, verifique o caminho ou faça o upload.")
if not os.path.exists(PATH_MICRORREGIAO):
    print(f"ATENÇÃO: O arquivo '{PATH_MICRORREGIAO}' não foi encontrado. Por favor, verifique o caminho ou faça o upload.")
if not os.path.exists(PATH_REPASSE_FEDERAL):
    print(f"ATENÇÃO: O arquivo '{PATH_REPASSE_FEDERAL}' não foi encontrado. Por favor, verifique o caminho ou faça o upload.")
if not os.path.exists(PATH_OBITOS):
    print(f"ATENÇÃO: O arquivo '{PATH_OBITOS}' não foi encontrado. Por favor, verifique o caminho ou faça o upload.")


try:
    # Dados do Censo e Microrregiões
    censo_ms_original = pd.read_csv(PATH_CENSO, sep = ";", encoding = "utf-8-sig")
    microrregiao_ms = pd.read_csv(PATH_MICRORREGIAO, sep = ";", encoding = "latin1")
    if 'ï»¿codmun7' in microrregiao_ms.columns:
        microrregiao_ms.rename(columns={'ï»¿codmun7': 'codmun7'}, inplace=True)

    # Carregar dados de repasse federal de fevereiro/2025
    df_repasse_federal = pd.read_excel(PATH_REPASSE_FEDERAL, sheet_name = "total")

    # Carregar dados de óbitos
    df_obitos_aps = pd.read_excel(PATH_OBITOS)

except FileNotFoundError as e:
    print(f"Erro ao carregar o arquivo: {e}. Certifique-se de que os caminhos estão corretos e os arquivos foram carregados no Colab.")
    exit()


# --- Célula 3: Preparação e Tratamento dos Dados do Censo e Criação de censo_com_porcentagens ---

# Verifica se os DataFrames necessários existem
if 'censo_ms_original' not in locals() or censo_ms_original.empty:
    print("ERRO: 'censo_ms_original' não carregado ou está vazio. Por favor, verifique a Célula 2.")
    exit()
if 'microrregiao_ms' not in locals() or microrregiao_ms.empty:
    print("ERRO: 'microrregiao_ms' não carregado ou está vazio. Por favor, verifique a Célula 2.")
    exit()

# Preparar dados do Censo
censo_ms_selecionado = censo_ms_original[[
    "CD_MUN",
    "NM_MUN",
    "V01400", # Quantidade total de residentes no município
    "V01316", # Quantidade de pessoas Quilombolas
    "V01321"  # Quantidade de pessoas Indígenas
]].copy()

censo_ms_selecionado.rename(columns = {"CD_MUN": "codmun7"}, inplace = True)
censo_ms_selecionado["codmun7"] = censo_ms_selecionado["codmun7"].astype(str).str.zfill(7)

# Preparar dados de Microrregiões
microrregiao_ms["codmun7"] = microrregiao_ms["codmun7"].astype(str).str.zfill(7)

# Juntar Censo e Microrregiões
censo_final_com_microrregiao = pd.merge(
    censo_ms_selecionado,
    microrregiao_ms[["codmun7", "nova_micro_proposta_ms"]],
    on = "codmun7",
    how = "left"
)

# Calcular porcentagens e criar censo_com_porcentagens
censo_com_porcentagens = censo_final_com_microrregiao.copy()
censo_com_porcentagens["porcentagem_indigena"] = np.where(
    censo_com_porcentagens["V01400"] > 0,
    (censo_com_porcentagens["V01321"] / censo_com_porcentagens["V01400"]) * 100,
    0
)
censo_com_porcentagens["porcentagem_quilombola"] = np.where(
    censo_com_porcentagens["V01400"] > 0,
    (censo_com_porcentagens["V01316"] / censo_com_porcentagens["V01400"]) * 100,
    0
)


# --- Célula 4: Preparação dos Dados de Repasse Federal e Criação de dados_principais (VERSÃO FINAL) ---

# Verifica se os DataFrames necessários existem antes de processar
if 'df_repasse_federal' not in locals() or df_repasse_federal.empty:
    print("ERRO: 'df_repasse_federal' não carregado ou está vazio. Por favor, verifique a Célula 2.")
    exit()
if 'censo_com_porcentagens' not in locals() or censo_com_porcentagens.empty:
    print("ERRO: 'censo_com_porcentagens' não criado ou está vazio. Por favor, execute a Célula 3 primeiro.")
    exit()
if 'df_obitos_aps' not in locals() or df_obitos_aps.empty:
    print("ERRO: 'df_obitos_aps' não carregado ou está vazio. Por favor, verifique a Célula 2.")
    exit()


# --- PARTE 1: Criação do DataFrame `dados_principais` (para o dashboard de cofinanciamento) ---
df_repasse_federal_limpo = df_repasse_federal.copy()

if 'codmun7' not in df_repasse_federal_limpo.columns:
    found_col_for_codmun7_repasse = None
    possible_code_cols_repasse = [col for col in df_repasse_federal_limpo.columns if "cod" in col.lower() or "municipio" in col.lower() or "ibge" in col.lower()]

    for col_name in possible_code_cols_repasse:
        if ("cod" in col_name.lower() and "mun" in col_name.lower()) or \
           ("cod" in col_name.lower() and "municipio" in col_name.lower()) or \
           ("ibge" in col_name.lower() and "codigo" in col_name.lower()) or \
           (col_name.lower() == "ibge"):
            found_col_for_codmun7_repasse = col_name
            break

    if found_col_for_codmun7_repasse:
        df_repasse_federal_limpo.rename(columns={found_col_for_codmun7_repasse: 'codmun7'}, inplace=True)
    else:
        print("ERRO CRÍTICO: Coluna 'codmun7' ou similar não encontrada em 'df_repasse_federal'. Por favor, verifique o nome da coluna no arquivo Excel (sheet 'total') ou os dados.")
        df_repasse_federal_limpo = pd.DataFrame(columns=['codmun7', 'cofinanciamento_federal_total_mensal'])

if 'codmun7' in df_repasse_federal_limpo.columns:
    df_repasse_federal_limpo["codmun7"] = df_repasse_federal_limpo["codmun7"].astype(str).str.zfill(7)
else:
    print("Aviso: 'codmun7' ainda não está presente em df_repasse_federal_limpo após tentativas de renomear. Verifique os dados.")

for col in df_repasse_federal_limpo.columns:
    if col not in ["codmun7", "IBGE"]:
        if df_repasse_federal_limpo[col].dtype == 'object':
            df_repasse_federal_limpo[col] = df_repasse_federal_limpo[col].astype(str).str.replace('.', '').str.replace(',', '.', regex=False)
            df_repasse_federal_limpo[col] = pd.to_numeric(df_repasse_federal_limpo[col], errors='coerce')
        elif not pd.api.types.is_numeric_dtype(df_repasse_federal_limpo[col]):
            df_repasse_federal_limpo[col] = pd.to_numeric(df_repasse_federal_limpo[col], errors='coerce')

cols_to_sum = [col for col in df_repasse_federal_limpo.columns if col not in ["codmun7", "IBGE"] and pd.api.types.is_numeric_dtype(df_repasse_federal_limpo[col])]
df_repasse_federal_limpo["cofinanciamento_federal_total_mensal"] = df_repasse_federal_limpo[cols_to_sum].sum(axis=1, skipna=True)
df_repasse_federal_limpo = df_repasse_federal_limpo[["codmun7", "cofinanciamento_federal_total_mensal"]]

# AQUI É ONDE `dados_principais` É DEFINIDO APENAS COM CENSO E REPASSE FEDERAL
dados_principais = pd.merge(
    censo_com_porcentagens,
    df_repasse_federal_limpo,
    on = "codmun7",
    how = "left"
)
dados_principais["cofinanciamento_federal_total_mensal"] = dados_principais["cofinanciamento_federal_total_mensal"].fillna(0)


# --- PARTE 2: Criação do DataFrame `df_regressao_base` (para a análise de regressão de mortalidade) ---
df_obitos_aps_limpo = df_obitos_aps.copy()

if 'codmun7' not in df_obitos_aps_limpo.columns:
    found_col_for_codmun7_obitos = None
    possible_code_cols_obitos = [col for col in df_obitos_aps_limpo.columns if "cod" in col.lower() or "municipio" in col.lower() or "ibge" in col.lower() or "id_munic" in col.lower()]

    for col_name in possible_code_cols_obitos:
        if ("cod" in col_name.lower() and "mun" in col_name.lower()) or \
           ("ibge" in col_name.lower()) or \
           ("codigo" in col_name.lower() and "municipio" in col_name.lower()) or \
           ("id_munic" in col_name.lower()):
            found_col_for_codmun7_obitos = col_name
            break
    if found_col_for_codmun7_obitos is None:
        for col_name in df_obitos_aps_limpo.columns:
            if col_name.lower() in ['municipio', 'código', 'cod'] and pd.api.types.is_numeric_dtype(df_obitos_aps_limpo[col_name]):
                found_col_for_codmun7_obitos = col_name
                break

    if found_col_for_codmun7_obitos:
        df_obitos_aps_limpo.rename(columns={found_col_for_codmun7_obitos: 'codmun7'}, inplace=True)
        print(f"ATENÇÃO: Coluna '{found_col_for_codmun7_obitos}' em 'df_obitos_aps' foi renomeada para 'codmun7'.")
    else:
        print("ERRO CRÍTICO: Coluna 'codmun7' ou similar não encontrada em 'df_obitos_aps'. Por favor, verifique o nome da coluna no arquivo Excel ou os dados.")
        df_obitos_aps_limpo['codmun7'] = np.nan

if 'codmun7' in df_obitos_aps_limpo.columns:
    df_obitos_aps_limpo["codmun7"] = df_obitos_aps_limpo["codmun7"].astype(str).str.zfill(7)
else:
    print("Aviso: 'codmun7' ainda não está presente em df_obitos_aps_limpo após tentativas de renomear. Mesclagem pode falhar ou conter NaNs.")


# Criação do DataFrame df_regressao_base: mescla apenas o necessário do censo com os dados de óbitos
df_regressao_base = pd.merge(
    censo_com_porcentagens[['codmun7', 'NM_MUN', 'V01400']], # Apenas as colunas necessárias do censo
    df_obitos_aps_limpo,
    on="codmun7",
    how="left"
)

# Limpeza e conversão robusta para colunas numéricas de df_regressao_base
columns_to_clean_and_convert = {
    'n_obitos_csap': int,
    'tx_mort_csap': float,
    'tx_mort_csap_aj_oms': float,
    'desp_tot_saude_pc_mun': float,
    'ano': int
}

for col, dtype in columns_to_clean_and_convert.items():
    if col in df_regressao_base.columns:
        df_regressao_base[col] = df_regressao_base[col].astype(str).str.strip()
        if dtype == float:
            df_regressao_base[col] = df_regressao_base[col].str.replace(',', '.', regex=False)
            df_regressao_base[col] = df_regressao_base[col].str.replace(r'[^\d.]', '', regex=True)
        elif dtype == int:
            df_regressao_base[col] = df_regressao_base[col].str.replace(r'[^\d]', '', regex=True)

        df_regressao_base[col] = pd.to_numeric(df_regressao_base[col], errors='coerce')

        if dtype == int:
            df_regressao_base[col] = df_regressao_base[col].fillna(0).astype(int)
        elif dtype == float:
            df_regressao_base[col] = df_regressao_base[col].fillna(0.0).astype(float)
    else:
        print(f"AVISO: Coluna '{col}' não encontrada em df_regressao_base. Criando coluna com valor 0/0.0 para df_regressao_base.")
        if dtype == int:
            df_regressao_base[col] = 0
        elif dtype == float:
            df_regressao_base[col] = 0.0

if 'ano' in df_regressao_base.columns:
    if (df_regressao_base['ano'] == -1).any():
        print("AVISO: Alguns valores de 'ano' foram preenchidos com -1 em df_regressao_base. Substituindo por 2010.")
        df_regressao_base['ano'] = df_regressao_base['ano'].replace(-1, 2010)

df_regressao_base = df_regressao_base[df_regressao_base['V01400'] > 0].copy()

if 'desp_tot_saude_pc_mun' in df_regressao_base.columns and 'n_obitos_csap' in df_regressao_base.columns:
    df_regressao_base.dropna(subset=['desp_tot_saude_pc_mun', 'n_obitos_csap'], inplace=True)
else:
    print("AVISO: Algumas colunas essenciais para regressão (desp_tot_saude_pc_mun, n_obitos_csap) não existem ou foram preenchidas com 0 em df_regressao_base.")

if 'desp_tot_saude_pc_mun' in df_regressao_base.columns:
    df_regressao_base['desp_tot_saude_pc_mun'] = df_regressao_base['desp_tot_saude_pc_mun'].clip(lower=0)
else:
    print("AVISO: A coluna 'desp_tot_saude_pc_mun' não existe em df_regressao_base para garantir valores não negativos.")

# IMPORTANTE: `dados_principais` não é sobrescrito aqui! Ele mantém os dados originais.


# --- Célula 5: Definição de Tabelas de IED e Vulnerabilidade ---
# Estas tabelas são estáticas e serão usadas pela função interativa.

# Tabela: IED de acordo com a portaria 3.493 10 de abril de 2024
df_ied_ms = pd.DataFrame({
    "codmun7": ["5000203", "5000252", "5000609", "5000708", "5000807", "5000856", "5000906", "5001003",
                "5001102", "5001243", "5001508", "5001904", "5002001", "5002100", "5002159", "5002209",
                "5002308", "5002407", "5002605", "5002704", "5002803", "5002902", "5002951", "5003108",
                "5003157", "5003207", "5003256", "5003306", "5003454", "5003488", "5003504", "5003702",
                "5003751", "5003801", "5003900", "5004007", "5004106", "5004304", "5004403", "5004502",
                "5004601", "5004700", "5004809", "5004908", "5005004", "5005103", "5005152", "5005202",
                "5005251", "5005400", "5005608", "5005681", "5005707", "5005806", "5006002", "5006200",
                "5006259", "5006275", "5006309", "5006358", "5006408", "5006606", "5006903", "5007109",
                "5007208", "5007307", "5007406", "5007505", "5007554", "5007695", "5007703", "5007802",
                "5007901", "5007935", "5007950", "5007976", "5008008", "5008305", "5008404"],
    "NM_MUN_IED": ["Água Clara", "Alcinópolis", "Amambai", "Anastácio", "Anaurilândia", "Angélica", "Antônio João", "Aparecida do Taboado",
                    "Aquidauana", "Aral Moreira", "Bandeirantes", "Bataguassu", "Batayporã", "Bela Vista", "Bodoquena", "Bonito",
                    "Brasilândia", "Caarapó", "Camapuã", "Campo Grande", "Caracol", "Cassilândia", "Chapadão do Sul", "Corguinho",
                    "Coronel Sapucaia", "Corumbá", "Costa Rica", "Coxim", "Deodápolis", "Dois Irmãos do Buriti", "Douradina", "Dourados",
                    "Eldorado", "Fátima do Sul", "Figueirão", "Glória de Dourados", "Guia Lopes da Laguna", "Iguatemi", "Inocência", "Itaporã",
                    "Itaquiraí", "Ivinhema", "Japorã", "Jaraguari", "Jardim", "Jateí", "Juti", "Ladário",
                    "Laguna Carapã", "Maracaju", "Miranda", "Mundo Novo", "Naviraí", "Nioaque", "Nova Alvorada do Sul", "Nova Andradina",
                    "Novo Horizonte do Sul", "Paraíso das Águas", "Paranaíba", "Paranhos", "Pedro Gomes", "Ponta Porã", "Porto Murtinho", "Ribas do Rio Pardo",
                    "Rio Brilhante", "Rio Negro", "Rio Verde de Mato Grosso", "Rochedo", "Santa Rita do Pardo", "São Gabriel do Oeste", "Sete Quedas", "Selvíria",
                    "Sidrolândia", "Sonora", "Tacuru", "Taquarussu", "Terenos", "Três Lagoas", "Vicentina"],
    "Pop_IBGE_IED": [16741, 4537, 39325, 24114, 7653, 10729, 9303, 27674,
                      46803, 10748, 7940, 23031, 10712, 21613, 8567, 23659,
                      11579, 30612, 13583, 898100, 5036, 20988, 30993, 4783,
                      14289, 96268, 26037, 32151, 13663, 11100, 5578, 243367,
                      11386, 20609, 3539, 10444, 9940, 13808, 8404, 24137,
                      19423, 27821, 8148, 7139, 23981, 3586, 6729, 21522,
                      6799, 45047, 25536, 19193, 50457, 13220, 21822, 48563,
                      4721, 5510, 40957, 12921, 6941, 92017, 12859, 23150,
                      37601, 4841, 19818, 5199, 7027, 29579, 10994, 8142,
                      47118, 14516, 10808, 3625, 17652, 132147, 6336],
    "Faixa_Porte_IBGE": ["1-Até 20mil hab.", "1-Até 20mil hab.", "2-Acima de 20mil até 50mil hab.", "2-Acima de 20mil até 50mil hab.",
                          "1-Até 20mil hab.", "1-Até 20mil hab.", "1-Até 20mil hab.", "2-Acima de 20mil até 50mil hab.",
                          "2-Acima de 20mil até 50mil hab.", "1-Até 20mil hab.", "1-Até 20mil hab.", "2-Acima de 20mil até 50mil hab.",
                          "1-Até 20mil hab.", "2-Acima de 20mil até 50mil hab.", "1-Até 20mil hab.", "2-Acima de 20mil até 50mil hab.",
                          "1-Até 20mil hab.", "2-Acima de 20mil até 50mil hab.", "1-Até 20mil hab.", "4-Acima de 100mil hab.",
                          "1-Até 20mil hab.", "2-Acima de 20mil até 50mil hab.", "2-Acima de 20mil até 50mil hab.", "1-Até 20mil hab.",
                          "1-Até 20mil hab.", "3-Acima de 50mil até 100mil hab.", "2-Acima de 20mil até 50mil hab.", "2-Acima de 20mil até 50mil hab.",
                          "1-Até 20mil hab.", "1-Até 20mil hab.", "1-Até 20mil hab.", "4-Acima de 100mil hab.",
                          "1-Até 20mil hab.", "2-Acima de 20mil até 50mil hab.", "1-Até 20mil hab.", "1-Até 20mil hab.",
                          "1-Até 20mil hab.", "1-Até 20mil hab.", "1-Até 20mil hab.", "2-Acima de 20mil até 50mil hab.",
                          "1-Até 20mil hab.", "2-Acima de 20mil até 50mil hab.", "1-Até 20mil hab.", "1-Até 20mil hab.",
                          "2-Acima de 20mil até 50mil hab.", "1-Até 20mil hab.", "1-Até 20mil hab.", "2-Acima de 20mil até 50mil hab.",
                          "1-Até 20mil hab.", "2-Acima de 20mil até 50mil hab.", "2-Acima de 20mil até 50mil hab.", "1-Até 20mil hab.",
                          "3-Acima de 50mil até 100mil hab.", "1-Até 20mil hab.", "2-Acima de 20mil até 50mil hab.", "2-Acima de 20mil até 50mil hab.",
                          "1-Até 20mil hab.", "1-Até 20mil hab.", "2-Acima de 20mil até 50mil hab.", "1-Até 20mil hab.",
                          "1-Até 20mil hab.", "3-Acima de 50mil até 100mil hab.", "1-Até 20mil hab.", "2-Acima de 20mil até 50mil hab.",
                          "2-Acima de 20mil até 50mil hab.", "1-Até 20mil hab.", "1-Até 20mil hab.", "1-Até 20mil hab.",
                          "1-Até 20mil hab.", "2-Acima de 20mil até 50mil hab.", "1-Até 20mil hab.", "1-Até 20mil hab.",
                          "2-Acima de 20mil até 50mil hab.", "1-Até 20mil hab.", "1-Até 20mil hab.", "1-Até 20mil hab.",
                          "1-Até 20mil hab.", "4-Acima de 100mil hab.", "1-Até 20mil hab."],
    "IED": [3, 3, 3, 3, 3, 3, 3, 4,
            3, 3, 3, 4, 3, 3, 3, 4,
            3, 2, 3, 4, 3, 4, 4, 3,
            2, 3, 4, 4, 3, 3, 3, 4,
            3, 4, 3, 3, 3, 3, 3, 2,
            3, 4, 1, 3, 4, 3, 2, 3,
            3, 4, 2, 3, 4, 2, 3, 4,
            3, 3, 4, 2, 3, 3, 3, 3,
            3, 3, 3, 3, 2, 4, 3, 3,
            3, 3, 1, 3, 3, 4, 3]
})
df_ied_ms["codmun7"] = df_ied_ms["codmun7"].astype(str).str.zfill(7)


# Tabela: Valores do Componente de Desempenho no Acompanhamento de Populações Vulneráveis (Estadual)
df_valores_desempenho_vulnerabilidade_estadual = pd.DataFrame({
    "Classificacao_Desempenho_Acomp_Vulneravel": [
        "Ótimo Acompanhamento (Escore >= 150)",
        "Bom Acompanhamento (Escore 100-149)",
        "Suficiente Acompanhamento (Escore 50-99)",
        "Regular Acompanhamento (Escore < 50)"
    ],
    "Valor_Desempenho_Acomp_Vulneravel_por_Municipio": [5000, 3000, 1000, 0]
})
# --- Célula 6: Finalização da Preparação de Dados (IED e Escore de Vulnerabilidade) ---

# Adicionar o IED dos municípios aos dados principais
# Verifica se dados_principais foi criado na Célula 4
if 'dados_principais' not in locals() or dados_principais.empty:
    print("ERRO: 'dados_principais' não criado ou está vazio. Por favor, verifique a Célula 4.")
    exit()

# Garante que df_ied_ms está disponível
if 'df_ied_ms' not in locals() or df_ied_ms.empty:
    print("ERRO: 'df_ied_ms' não criado ou está vazio. Por favor, verifique a Célula 5.")
    exit()

# Check if 'IED' column already exists in dados_principais before merging
if 'IED' not in dados_principais.columns:
    dados_principais = pd.merge(
        dados_principais,
        df_ied_ms[["codmun7", "IED"]],
        on = "codmun7",
        how = "left"
    )
    dados_principais["IED"] = dados_principais["IED"].fillna(3) # Tratar IED NA como um estrato padrão se algum município não for encontrado
else:
    pass # Coluna 'IED' já existe, não fazer nada.


# Calcular o escore semestral de vulnerabilidade (inicialmente, sem os pesos dos sliders)
# NOTA: As variáveis 'porcentagem_flutuante_imigrante' e 'porcentagem_sus_dependente'
# são SIMULADAS aqui para a base inicial. Na aplicação real, elas precisariam ser importadas
# de dados reais de acompanhamento ou bases de dados.
np.random.seed(42) # Para reprodutibilidade da simulação inicial

dados_principais["porcentagem_flutuante_imigrante"] = np.random.uniform(0, 0.05, size = len(dados_principais)) * 100
dados_principais["porcentagem_sus_dependente"] = np.random.uniform(70, 100, size = len(dados_principais))

# Os pesos iniciais são os mesmos do `value` dos sliders
dados_principais["escore_semestral_vulnerabilidade"] = (
    (dados_principais["porcentagem_indigena"] * 1.5) +
    (dados_principais["porcentagem_quilombola"] * 1.2) +
    (dados_principais["porcentagem_flutuante_imigrante"] * 2.0) +
    (dados_principais["porcentagem_sus_dependente"] * 0.5)
)
dados_principais["escore_semestral_vulnerabilidade"] = dados_principais["escore_semestral_vulnerabilidade"].clip(upper = 200)


# --- Célula 7: Função Principal Interativa do Dashboard ---
# Esta função contém toda a lógica de cálculo e visualização que será atualizada pelos widgets.

# Função auxiliar para formatar valores monetários (movida para dentro da função interativa para modularidade)
def format_currency(value):
    return f"R${value:,.0f}".replace(",", "X").replace(".", ",").replace("X", ".")


def run_dashboard(orcamento_anual_total, reserva_equidade_anual,
                  peso_indigena, peso_quilombola, peso_flutuante_imigrante, peso_sus_dependente,
                  random_seed_vulnerabilidade,
                  percentual_ied1, percentual_ied2, percentual_ied3, percentual_ied4): # Adicionados novos parâmetros

    # Garantir a reprodutibilidade da simulação com a seed do widget para as porcentagens simuladas
    np.random.seed(random_seed_vulnerabilidade)

    # Cria uma cópia de dados_principais que já possui IED e as porcentagens
    dados_principais_temp = dados_principais.copy()

    # Re-calcular APENAS o escore de vulnerabilidade com os NOVOS pesos dos sliders
    # As porcentagens flutuante/imigrante e SUS dependente são geradas aqui a cada atualização
    # para que a random_seed_vulnerabilidade tenha efeito.
    dados_principais_temp["porcentagem_flutuante_imigrante"] = np.random.uniform(0, 0.05, size = len(dados_principais_temp)) * 100
    dados_principais_temp["porcentagem_sus_dependente"] = np.random.uniform(70, 100, size = len(dados_principais_temp))


    dados_principais_temp["escore_semestral_vulnerabilidade"] = (
        (dados_principais_temp["porcentagem_indigena"] * peso_indigena) +
        (dados_principais_temp["porcentagem_quilombola"] * peso_quilombola) +
        (dados_principais_temp["porcentagem_flutuante_imigrante"] * peso_flutuante_imigrante) +
        (dados_principais_temp["porcentagem_sus_dependente"] * peso_sus_dependente) # Corrigido: usar dados_principais_temp
    )
    dados_principais_temp["escore_semestral_vulnerabilidade"] = dados_principais_temp["escore_semestral_vulnerabilidade"].clip(upper = 200)


    # Função para calcular o cofinanciamento estadual (Modelo Híbrido)
    # Esta função agora recebe o DataFrame com o IED e o escore de vulnerabilidade já calculado
    def calcular_cofinanciamento_estadual_interativo(df_base_municipios_interativo):
        df_calculo_interativo = df_base_municipios_interativo.copy()

        # Componente 1: Acréscimo Percentual Baseado no IED (proporcional ao Federal REAL)
        # USA OS NOVOS PARÂMETROS percentual_ied1, percentual_ied2, etc.
        percentuais_acrescimo_estadual = pd.DataFrame({
            "IED": [1, 2, 3, 4],
            "percentual": [percentual_ied1, percentual_ied2, percentual_ied3, percentual_ied4]
        })

        df_calculo_interativo = pd.merge(
            df_calculo_interativo,
            percentuais_acrescimo_estadual,
            on = "IED",
            how = "left"
        )
        df_calculo_interativo["cofinanciamento_estadual_ied_mensal"] = df_calculo_interativo["cofinanciamento_federal_total_mensal"] * df_calculo_interativo["percentual"]
        df_calculo_interativo["cofinanciamento_estadual_ied_mensal"] = df_calculo_interativo["cofinanciamento_estadual_ied_mensal"].fillna(0)

        # Componente 2: Incentivo ao Desempenho no Acompanhamento de Populações Vulneráveis
        df_calculo_interativo["Classificacao_Desempenho_Acomp_Vulneravel"] = pd.cut(
            df_calculo_interativo["escore_semestral_vulnerabilidade"],
            bins = [-np.inf, 50, 99, 149, np.inf],
            labels = [
                "Regular Acompanhamento (Escore < 50)",
                "Suficiente Acompanhamento (Escore 50-99)",
                "Bom Acompanhamento (Escore 100-149)",
                "Ótimo Acompanhamento (Escore >= 150)"
            ],
            right = False
        )

        df_calculo_interativo = pd.merge(
            df_calculo_interativo,
            df_valores_desempenho_vulnerabilidade_estadual, # df_valores_desempenho_vulnerabilidade_estadual é global
            on = "Classificacao_Desempenho_Acomp_Vulneravel",
            how = "left"
        )
        df_calculo_interativo["valor_componente_desempenho_vulnerabilidade_mensal"] = df_calculo_interativo["Valor_Desempenho_Acomp_Vulneravel_por_Municipio"].fillna(0)
        # Manter 'Classificacao_Desempenho_Acomp_Vulneravel' para o Gráfico 2.
        # Não dropar 'Valor_Desempenho_Acomp_Vulneravel_por_Municipio' aqui se for usá-la depois
        # df_calculo_interativo.drop(columns = ["Valor_Desempenho_Acomp_Vulneravel_por_Municipio"], inplace = True)

        # Soma do Cofinanciamento Estadual Total Mensal por Município
        df_calculo_interativo["cofinanciamento_estadual_final_mensal"] = df_calculo_interativo["cofinanciamento_estadual_ied_mensal"] + \
                                                                         df_calculo_interativo["valor_componente_desempenho_vulnerabilidade_mensal"]

        return df_calculo_interativo

    # Cálculo do Cenário Principal com os parâmetros interativos
    dados_cenario_principal_interativo = calcular_cofinanciamento_estadual_interativo(
        df_base_municipios_interativo = dados_principais_temp
    )

    total_estadual_necessario_mensal_principal = dados_cenario_principal_interativo["cofinanciamento_estadual_final_mensal"].sum()
    total_estadual_necessario_anual_principal = total_estadual_necessario_mensal_principal * 12

    orcamento_anual_disp_distribuicao = orcamento_anual_total - reserva_equidade_anual

    fator_ajuste_orcamentario_principal = 1
    if total_estadual_necessario_anual_principal > orcamento_anual_disp_distribuicao:
        fator_ajuste_orcamentario_principal = orcamento_anual_disp_distribuicao / total_estadual_necessario_anual_principal

    # Apenas esta coluna é influenciada pelo fator de ajuste para a exibição na tabela e painel "ajustado"
    dados_cenario_principal_interativo["cofinanciamento_estadual_ajustado_mensal"] = dados_cenario_principal_interativo["cofinanciamento_estadual_final_mensal"] * fator_ajuste_orcamentario_principal
    dados_cenario_principal_interativo["cofinanciamento_estadual_ajustado_anual"] = dados_cenario_principal_interativo["cofinanciamento_estadual_ajustado_mensal"] * 12

    # Preparar dataframe final para análise e exportação
    df_analise_interativo = dados_cenario_principal_interativo.copy()

    # Renomeando colunas para clareza e uso consistente
    df_analise_interativo = df_analise_interativo.rename(columns={
        "cofinanciamento_federal_total_mensal": "cofinanciamento_federal_total_mensal_real",
        "cofinanciamento_estadual_final_mensal": "cofinanciamento_estadual_final_mensal_proposto",
        "percentual": "percentual_acrescimo_ied_estadual",
        # Renomeia a coluna do componente de vulnerabilidade para um nome mais amigável e consistente
        "valor_componente_desempenho_vulnerabilidade_mensal": "valor_componente_vulnerabilidade_mensal_estadual"
    })

    # Coluna para ordenação do gráfico (Federal + Estadual Proposto)
    df_analise_interativo["total_recebimento_mensal_para_grafico"] = df_analise_interativo["cofinanciamento_federal_total_mensal_real"] + df_analise_interativo["cofinanciamento_estadual_final_mensal_proposto"]
    df_analise_interativo["total_recebimento_anual_para_grafico"] = df_analise_interativo["total_recebimento_mensal_para_grafico"] * 12

    # Nova coluna para o total de recebimento com o ajuste estadual (Federal Real + Estadual Ajustado)
    df_analise_interativo["total_recebimento_mensal_com_ajuste_estadual"] = df_analise_interativo["cofinanciamento_federal_total_mensal_real"] + df_analise_interativo["cofinanciamento_estadual_ajustado_mensal"]
    df_analise_interativo["total_recebimento_anual_com_ajuste_estadual"] = df_analise_interativo["total_recebimento_mensal_com_ajuste_estadual"] * 12


    df_analise_interativo = df_analise_interativo[[
        "codmun7", "NM_MUN", "V01400", "IED", "escore_semestral_vulnerabilidade",
        "porcentagem_indigena", "porcentagem_quilombola",
        "porcentagem_flutuante_imigrante", "porcentagem_sus_dependente",
        "cofinanciamento_federal_total_mensal_real",
        "percentual_acrescimo_ied_estadual",
        "cofinanciamento_estadual_ied_mensal",
        "valor_componente_vulnerabilidade_mensal_estadual", # Usar o novo nome
        "cofinanciamento_estadual_final_mensal_proposto", # Valor proposto antes do ajuste
        "cofinanciamento_estadual_ajustado_mensal", # Valor ajustado
        "cofinanciamento_estadual_ajustado_anual",
        "total_recebimento_mensal_para_grafico", # Para ordenação do gráfico e soma no gráfico
        "total_recebimento_anual_para_grafico",
        "total_recebimento_mensal_com_ajuste_estadual", # Novo total com o ajuste
        "total_recebimento_anual_com_ajuste_estadual" # Novo total anual com o ajuste
    ]]

    # Limpar saída anterior para exibir os novos gráficos e dados
    clear_output(wait=True)

    # --- Configuração das Abas ---
    tab_output_painel = widgets.Output()
    tab_output_graficos = widgets.Output()
    tab_output_tabela = widgets.Output()
    tab_output_projecao_mortalidade = widgets.Output()
    # tab_output_regressao = widgets.Output() # Removido para simplificar

    # Conteúdo da Aba 1: Resumo Financeiro
    with tab_output_painel:
        print("--- Painel de Custos para o Estado ---")
        print(f"Orçamento Anual Total (Definido): {format_currency(orcamento_anual_total)}")
        print(f"Reserva de Equidade Anual (Definida): {format_currency(reserva_equidade_anual)}")
        print(f"Orçamento Anual Disponível para Distribuição: {format_currency(orcamento_anual_disp_distribuicao)}")
        print(f"Cofinanciamento FEDERAL Total Mensal REAL: {format_currency(df_analise_interativo['cofinanciamento_federal_total_mensal_real'].sum())}")
        print(f"Cofinanciamento ESTADUAL Proposto (Híbrido) Mensal: {format_currency(df_analise_interativo['cofinanciamento_estadual_final_mensal_proposto'].sum())}")
        print(f"Cofinanciamento ESTADUAL Proposto (Híbrido) Anual: {format_currency(df_analise_interativo['cofinanciamento_estadual_final_mensal_proposto'].sum() * 12)}")
        if fator_ajuste_orcamentario_principal < 1:
            print(f"ATENÇÃO: Orçamento estadual necessário excede o disponível. Fator de ajuste proporcional aplicado: {fator_ajuste_orcamentario_principal:.4f}.")
        print(f"Total Mensal Cofinanciamento Estadual AJUSTADO: {format_currency(df_analise_interativo['cofinanciamento_estadual_ajustado_mensal'].sum())}")
        print(f"Total Anual Cofinanciamento Estadual AJUSTADO: {format_currency(df_analise_interativo['cofinanciamento_estadual_ajustado_anual'].sum())}")
        print(f"TOTAL DE CUSTO ANUAL PARA O ESTADO (Federal Real + Estadual Ajustado): {format_currency(df_analise_interativo['total_recebimento_anual_com_ajuste_estadual'].sum())}\n")

        if fator_ajuste_orcamentario_principal < 1:
            display(HTML(f"""
                <div style='background-color:#ffe5e5; padding:10px; border-left:5px solid red; margin-bottom:15px;'>
                    <strong style='color:#990000;'>⚠️ O orçamento anual projetado ({format_currency(df_analise_interativo['cofinanciamento_estadual_final_mensal_proposto'].sum() * 12)}) excede o disponível ({format_currency(orcamento_anual_disp_distribuicao)}).</strong><br>
                    Os valores mensais de cofinanciamento estadual foram <strong>ajustados proporcionalmente</strong> com fator de ajuste de <strong>{fator_ajuste_orcamentario_principal:.4f}</strong>.
                </div>
            """))

    # Conteúdo da Aba 2: Visualizações (Gráficos)
    with tab_output_graficos:
        municipios_ordenados = df_analise_interativo.sort_values(
            by="total_recebimento_mensal_para_grafico", ascending=False
        )
        municipios_long = municipios_ordenados[[
            "NM_MUN",
            "cofinanciamento_federal_total_mensal_real",
            "cofinanciamento_estadual_final_mensal_proposto"
        ]].melt(
            id_vars="NM_MUN",
            var_name="Tipo_Cofinanciamento",
            value_name="Valor"
        )
        municipios_long["Tipo_Cofinanciamento"] = municipios_long["Tipo_Cofinanciamento"].replace({
            "cofinanciamento_federal_total_mensal_real": "Cofinanciamento Federal",
            "cofinanciamento_estadual_final_mensal_proposto": "Cofinanciamento Estadual Proposto"
        })

        plt.figure(figsize=(12, len(municipios_ordenados) * 0.22))
        sns.barplot(
            x="Valor",
            y="NM_MUN",
            hue="Tipo_Cofinanciamento",
            data=municipios_long,
            palette=["#ffc20a", "#ffd368"], # Cores principais e auxiliares
            orient="h",
            dodge=True
        )
        plt.title("Cofinanciamento Federal e Estadual Proposto por Município (Mensal)", fontsize=15)
        plt.xlabel("Valor Mensal (R$)", fontsize=12)
        plt.ylabel("Município", fontsize=12)
        plt.ticklabel_format(style='plain', axis='x')
        plt.grid(axis='x', linestyle='--', alpha=0.7)
        plt.xticks(fontsize=9)
        plt.yticks(fontsize=7)
        plt.legend(title="Tipo de Cofinanciamento", fontsize=9, title_fontsize=10)
        plt.tight_layout()
        plt.show()

        df_desempenho = dados_cenario_principal_interativo.groupby('Classificacao_Desempenho_Acomp_Vulneravel')['NM_MUN'].count().reset_index()
        df_desempenho.columns = ['Classificacao', 'Numero_Municipios']
        ordem_classificacao = [
            "Ótimo Acompanhamento (Escore >= 150)", "Bom Acompanhamento (Escore 100-149)",
            "Suficiente Acompanhamento (Escore 50-99)", "Regular Acompanhamento (Escore < 50)"
        ]
        df_desempenho['Classificacao'] = pd.Categorical(df_desempenho['Classificacao'], categories=ordem_classificacao, ordered=True)
        df_desempenho = df_desempenho.sort_values('Classificacao')

        plt.figure(figsize=(10, 6))
        sns.barplot(x='Classificacao', y='Numero_Municipios', data=df_desempenho, palette=['#f29100', '#f36f23', '#ffd368', '#ffc20a']) # Cores variadas da paleta
        plt.title("Distribuição dos Municípios por Desempenho em Vulnerabilidade", fontsize=15)
        plt.xlabel("Classificação de Desempenho", fontsize=12)
        plt.ylabel("Número de Municípios", fontsize=12)
        plt.xticks(rotation=45, ha='right', fontsize=10)
        plt.yticks(fontsize=10)
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        plt.tight_layout()
        plt.show()

        df_ied_acrescimo = dados_cenario_principal_interativo.groupby('IED')['NM_MUN'].count().reset_index()
        df_ied_acrescimo.columns = ['IED', 'Numero_Municipios']

        plt.figure(figsize=(8, 5))
        sns.barplot(x='IED', y='Numero_Municipios', data=df_ied_acrescimo, palette=['#000000', '#f29100', '#ffc20a', '#ffd368']) # Cores da paleta
        plt.title("Distribuição dos Municípios por IED", fontsize=15)
        plt.xlabel("Índice de Equidade Descentralizada (IED)", fontsize=12)
        plt.ylabel("Número de Municípios", fontsize=12)
        plt.xticks(fontsize=10)
        plt.yticks(fontsize=10)
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        plt.tight_layout()
        plt.show()

    # Conteúdo da Aba 3: Tabela Detalhada
    with tab_output_tabela:
        print("\n--- Tabela Detalhada de Cofinanciamento Mensal por Município (R$) ---")

        tabela_detalhada = df_analise_interativo[[
            "NM_MUN", "IED", "valor_componente_vulnerabilidade_mensal_estadual",
            "cofinanciamento_estadual_final_mensal_proposto",
            "cofinanciamento_estadual_ajustado_mensal"
        ]].copy()

        for col in [
            "valor_componente_vulnerabilidade_mensal_estadual",
            "cofinanciamento_estadual_final_mensal_proposto",
            "cofinanciamento_estadual_ajustado_mensal"
        ]:
            if col in tabela_detalhada.columns:
                tabela_detalhada[col] = pd.to_numeric(tabela_detalhada[col], errors='coerce').fillna(0)
                tabela_detalhada[col] = tabela_detalhada[col].apply(lambda x: f"R$ {x:,.2f}".replace(",", "X").replace(".", ",").replace("X", "."))

        tabela_detalhada = tabela_detalhada.rename(columns={
            "NM_MUN": "Município", "IED": "IED",
            "valor_componente_vulnerabilidade_mensal_estadual": "Valor Vulnerabilidade (R$)",
            "cofinanciamento_estadual_final_mensal_proposto": "Estadual Proposto (R$)",
            "cofinanciamento_estadual_ajustado_mensal": "Estadual Ajustado (R$)"
        })
        display(HTML(tabela_detalhada.to_html(index=False)))

        # NOVO GRÁFICO: Composição do Cofinanciamento Estadual Proposto
        # Preparar dados para o gráfico
        df_composicao_cofin = dados_cenario_principal_interativo[[
            'NM_MUN',
            'valor_componente_desempenho_vulnerabilidade_mensal',
            'cofinanciamento_estadual_ied_mensal'
        ]].copy()

        # Renomear para clareza no gráfico
        df_composicao_cofin.rename(columns={
            'valor_componente_desempenho_vulnerabilidade_mensal': 'Componente Vulnerabilidade',
            'cofinanciamento_estadual_ied_mensal': 'Componente IED'
        }, inplace=True)

        # Converter para formato longo para barras empilhadas
        df_composicao_long = df_composicao_cofin.melt(
            id_vars='NM_MUN',
            var_name='Tipo de Componente',
            value_name='Valor'
        )

        # Ordenar os municípios pela soma total do cofinanciamento estadual proposto
        df_composicao_long_sorted = df_composicao_long.groupby('NM_MUN')['Valor'].sum().sort_values(ascending=False).index
        df_composicao_long['NM_MUN'] = pd.Categorical(df_composicao_long['NM_MUN'], categories=df_composicao_long_sorted, ordered=True)
        df_composicao_long.sort_values('NM_MUN', inplace=True)


        plt.figure(figsize=(12, len(df_composicao_long_sorted) * 0.25)) # Altura dinâmica
        sns.barplot(
            x='Valor',
            y='NM_MUN',
            hue='Tipo de Componente',
            data=df_composicao_long,
            palette={'Componente Vulnerabilidade': '#f36f23', 'Componente IED': '#ffc20a'}, # Cores definidas
            orient='h',
            dodge=False, # Essencial para barras empilhadas
            edgecolor='black' # Adiciona borda para melhor separação
        )
        plt.title('Composição do Cofinanciamento Estadual Proposto por Município (Mensal)', fontsize=15)
        plt.xlabel('Valor Mensal (R$)', fontsize=12)
        plt.ylabel('Município', fontsize=12)
        plt.ticklabel_format(style='plain', axis='x')
        plt.grid(axis='x', linestyle='--', alpha=0.7)
        plt.xticks(fontsize=9)
        plt.yticks(fontsize=7)
        plt.legend(title='Componente', fontsize=9, title_fontsize=10)
        plt.tight_layout()
        plt.show()

    # Conteúdo da Aba 4: Projeção da Mortalidade (OLS)
    with tab_output_projecao_mortalidade:
        print("--- Projeção da Mortalidade Média por CSAP (MS) ---")
        print("Baseado na tendência histórica da mortalidade ajustada de 2010-2023.")

        df_mortalidade_anual = df_regressao_base.groupby('ano')['tx_mort_csap_aj_oms'].mean().reset_index()
        df_mortalidade_anual = df_mortalidade_anual[(df_mortalidade_anual['ano'] >= 2010) & (df_mortalidade_anual['ano'] <= 2023)].copy()

        if not df_mortalidade_anual.empty and len(df_mortalidade_anual) > 1:
            try:
                X_mortalidade = sm.add_constant(df_mortalidade_anual['ano'])
                y_mortalidade = df_mortalidade_anual['tx_mort_csap_aj_oms']

                model_mortalidade = sm.OLS(y_mortalidade, X_mortalidade)
                results_mortalidade = model_mortalidade.fit()

                all_years_mortalidade = pd.DataFrame({'ano': range(2010, 2027)})
                X_predict_mortalidade = sm.add_constant(all_years_mortalidade['ano'])

                predictions_mortalidade = results_mortalidade.get_prediction(X_predict_mortalidade).summary_frame(alpha=0.05)
                predictions_mortalidade['ano'] = all_years_mortalidade['ano']

                plot_data_mortalidade = pd.merge(
                    df_mortalidade_anual,
                    predictions_mortalidade[['ano', 'mean', 'obs_ci_lower', 'obs_ci_upper']],
                    on='ano',
                    how='outer',
                    suffixes=('_observed', '_predicted')
                )

                plot_data_mortalidade['mean'] = plot_data_mortalidade['tx_mort_csap_aj_oms'].fillna(plot_data_mortalidade['mean'])


                plt.figure(figsize=(10, 6))

                sns.lineplot(
                    x='ano',
                    y='tx_mort_csap_aj_oms',
                    data=df_mortalidade_anual,
                    marker='o',
                    color='#f36f23', # Cor da linha histórica (laranja forte)
                    label='Mortalidade Observada (Média Anual)'
                )

                sns.lineplot(
                    x='ano',
                    y='mean',
                    data=plot_data_mortalidade,
                    color='#ffc20a', # Cor da linha de projeção (amarelo)
                    linestyle='--',
                    label='Tendência e Projeção (OLS)'
                )

                plt.fill_between(
                    plot_data_mortalidade['ano'],
                    plot_data_mortalidade['obs_ci_lower'],
                    plot_data_mortalidade['obs_ci_upper'],
                    color='#ffd368', # Cor do CI (amarelo claro)
                    alpha=0.3,
                    label='Intervalo de Confiança (95%)'
                )

                plt.axvline(x=2023.5, color='#000000', linestyle=':', label='Início Projeção (Após 2023)') # Preto para linha de projeção

                plt.title('Projeção da Mortalidade Média Ajustada por CSAP ao longo do Tempo', fontsize=15)
                plt.xlabel('Ano', fontsize=12)
                plt.ylabel('Mortalidade Ajustada (por 100.000 Hab.)', fontsize=12)
                plt.xticks(all_years_mortalidade['ano'].unique())
                plt.grid(True, linestyle='--', alpha=0.7, color='#cccccc') # Cinza claro para grid
                plt.legend(fontsize=10)
                plt.tight_layout()
                plt.show()

                print("\n--- Resultados da Regressão Linear para a Tendência da Mortalidade (Sumário) ---")
                display(HTML(results_mortalidade.summary().as_html()))

            except Exception as e:
                print(f"Erro ao executar a regressão linear para projeção de mortalidade: {e}")
                print("Verifique se há dados de mortalidade suficientes e válidos para a análise.")
        else:
            print("Dados de mortalidade insuficientes ou inválidos para realizar a projeção baseada em tendência temporal.")


    # Criação das Abas
    tab_widget = widgets.Tab()
    tab_widget.children = [tab_output_painel, tab_output_graficos, tab_output_tabela, tab_output_projecao_mortalidade]
    tab_widget.set_title(0, 'Resumo')
    tab_widget.set_title(1, 'Visualizações')
    tab_widget.set_title(2, 'Detalhamento')
    tab_widget.set_title(3, 'Projeção Mortalidade')

    display(tab_widget)


# --- Célula 8: Criação e Exibição dos Widgets Interativos (VERSÃO FINAL E LIMPA) ---

# Definir os widgets
slider_orcamento_anual = widgets.FloatSlider(
    value=30000000,
    min=10000000,
    max=50000000,
    step=1000000,
    description='R$:', # Descrição mais curta para o valor ser mais visível
    orientation='horizontal',
    readout=True,
    readout_format=',.0f', # Apenas o formato numérico, sem "R$"
    continuous_update=True,
    layout=widgets.Layout(width='320px') # Ajustar a largura para acomodar o label
)

slider_reserva_equidade = widgets.FloatSlider(
    value=1000000,
    min=0,
    max=5000000,
    step=100000,
    description='R$:', # Descrição mais curta
    orientation='horizontal',
    readout=True,
    readout_format=',.0f', # Apenas o formato numérico, sem "R$"
    continuous_update=True,
    layout=widgets.Layout(width='320px') # Ajustar a largura para acomodar o label
)

# Sliders para pesos do escore de vulnerabilidade
slider_peso_indigena = widgets.FloatSlider(
    value=1.5, min=0.0, max=5.0, step=0.1, description='P. Indíg.:', readout=True, layout=widgets.Layout(width='350px')
)
slider_peso_quilombola = widgets.FloatSlider(
    value=1.2, min=0.0, max=5.0, step=0.1, description='P. Quilo.:', readout=True, layout=widgets.Layout(width='350px')
)
slider_peso_flutuante_imigrante = widgets.FloatSlider(
    value=2.0, min=0.0, max=5.0, step=0.1, description='P. Pop. Flut./Imig.:', readout=True, layout=widgets.Layout(width='350px')
)
slider_peso_sus_dependente = widgets.FloatSlider(
    value=0.5, min=0.0, max=5.0, step=0.1, description='P. SUS Dep.:', readout=True, layout=widgets.Layout(width='350px')
)

slider_random_seed = widgets.IntSlider(
    value=42, min=1, max=1000, step=1, description='Seed Aleat. Vulnerab.:', readout=True, layout=widgets.Layout(width='350px')
)

# Sliders para os percentuais de IED
slider_percentual_ied1 = widgets.FloatSlider(
    value=0.20, min=0.0, max=0.5, step=0.01, description='IED 1 (% F.):', readout=True, readout_format='.2f', layout=widgets.Layout(width='350px')
)
slider_percentual_ied2 = widgets.FloatSlider(
    value=0.15, min=0.0, max=0.5, step=0.01, description='IED 2 (% F.):', readout=True, readout_format='.2f', layout=widgets.Layout(width='350px')
)
slider_percentual_ied3 = widgets.FloatSlider(
    value=0.10, min=0.0, max=0.5, step=0.01, description='IED 3 (% F.):', readout=True, readout_format='.2f', layout=widgets.Layout(width='350px')
)
slider_percentual_ied4 = widgets.FloatSlider(
    value=0.05, min=0.0, max=0.5, step=0.01, description='IED 4 (% F.):', readout=True, readout_format='.2f', layout=widgets.Layout(width='350px')
)

# Agrupamento para a primeira linha (Orçamento e Reserva)
orcamento_reserva_group = widgets.HBox([
    widgets.VBox([
        widgets.Label(value="Orçamento Anual Total:"),
        slider_orcamento_anual
    ], layout=widgets.Layout(align_items='flex-start')),
    widgets.VBox([
        widgets.Label(value="Reserva de Equidade Anual:"),
        slider_reserva_equidade
    ], layout=widgets.Layout(align_items='flex-start'))
], layout=widgets.Layout(justify_content='space-around', width='100%'))

# Agrupar os sliders de peso de vulnerabilidade
vulnerabilidade_sliders = widgets.VBox([
    widgets.Label(value="Pesos para o Escore de Vulnerabilidade:"),
    widgets.HBox([slider_peso_indigena]),
    widgets.HBox([slider_peso_quilombola]),
    widgets.HBox([slider_peso_flutuante_imigrante]),
    widgets.HBox([slider_peso_sus_dependente]),
    widgets.HBox([slider_random_seed])
], layout=widgets.Layout(align_items='flex-start'))

# Agrupar os sliders de percentual por IED
ied_sliders = widgets.VBox([
    widgets.Label(value="Acréscimo Percentual do Cofinanciamento Estadual por IED:"),
    widgets.HBox([slider_percentual_ied1]),
    widgets.HBox([slider_percentual_ied2]),
    widgets.HBox([slider_percentual_ied3]),
    widgets.HBox([slider_percentual_ied4])
], layout=widgets.Layout(align_items='flex-start'))

# Agrupar todos os controles
all_controls = widgets.VBox([
    orcamento_reserva_group,
    widgets.HBox([vulnerabilidade_sliders, ied_sliders])
], layout=widgets.Layout(align_items='flex-start'))


# Criar o widget de saída
output = widgets.Output()

# Função que será chamada quando os sliders mudarem
def on_value_change(change):
    with output:
        run_dashboard(
            orcamento_anual_total=slider_orcamento_anual.value,
            reserva_equidade_anual=slider_reserva_equidade.value,
            peso_indigena=slider_peso_indigena.value,
            peso_quilombola=slider_peso_quilombola.value,
            peso_flutuante_imigrante=slider_peso_flutuante_imigrante.value,
            peso_sus_dependente=slider_peso_sus_dependente.value,
            random_seed_vulnerabilidade=slider_random_seed.value,
            percentual_ied1=slider_percentual_ied1.value,
            percentual_ied2=slider_percentual_ied2.value,
            percentual_ied3=slider_percentual_ied3.value,
            percentual_ied4=slider_percentual_ied4.value
        )

# Vincular os sliders à função de atualização do dashboard
slider_orcamento_anual.observe(on_value_change, names='value')
slider_reserva_equidade.observe(on_value_change, names='value')
slider_peso_indigena.observe(on_value_change, names='value')
slider_peso_quilombola.observe(on_value_change, names='value')
slider_peso_flutuante_imigrante.observe(on_value_change, names='value')
slider_peso_sus_dependente.observe(on_value_change, names='value')
slider_random_seed.observe(on_value_change, names='value')
slider_percentual_ied1.observe(on_value_change, names='value')
slider_percentual_ied2.observe(on_value_change, names='value')
slider_percentual_ied3.observe(on_value_change, names='value')
slider_percentual_ied4.observe(on_value_change, names='value')


# Exibir os controles e o output
display(all_controls, output)

# Executar a função uma vez para exibir o dashboard inicial
with output:
    run_dashboard(
        orcamento_anual_total=slider_orcamento_anual.value,
        reserva_equidade_anual=slider_reserva_equidade.value,
        peso_indigena=slider_peso_indigena.value,
        peso_quilombola=slider_peso_quilombola.value,
        peso_flutuante_imigrante=slider_peso_flutuante_imigrante.value,
        peso_sus_dependente=slider_peso_sus_dependente.value,
        random_seed_vulnerabilidade=slider_random_seed.value,
        percentual_ied1=slider_percentual_ied1.value,
        percentual_ied2=slider_percentual_ied2.value,
        percentual_ied3=slider_percentual_ied3.value,
        percentual_ied4=slider_percentual_ied4.value
    )

Output()